In [1]:
import os
import pandas as pd
import tweepy
import random 
import numpy as np
import datetime
from datetime import date
import warnings
import requests


from PIL import Image
import requests
from io import BytesIO

# settings
pd.set_option('display.max_colwidth', None)

# time
today = date.today()
today_weekday = today.weekday()
now = datetime.datetime.now()
current_time = now.strftime("%H:%M:%S")
print("día de la semana hora local UTC =", today_weekday)
print("hoy es", today)
print("Hora actual en UTC =", current_time)

día de la semana hora local UTC = 1
hoy es 2022-08-23
Hora actual en UTC = 20:32:20


In [2]:
# environmental secrets when working locally
from dotenv import load_dotenv
try:
    load_dotenv()
    load_dotenv("../../../")
    print("envs loaded")
except:
    print("envs failed to load")
    pass

envs loaded


# Set up tweepy

In [3]:
#USING GITHUB ENVS

# test
BG1_TWITTER_APIKEY= os.environ["BG1_TWITTER_APIKEY"]
BG1_TWITTER_APIKEYSECRET = os.environ["BG1_TWITTER_APIKEYSECRET"]

BG1_TWITTER_BEARERTOKEN = os.environ["BG1_TWITTER_BEARERTOKEN"]
BG1_TWITTER_ACCESSTOKEN = os.environ["BG1_TWITTER_ACCESSTOKEN"]
BG1_TWITTER_ACCESSTOKENSECRET = os.environ["BG1_TWITTER_ACCESSTOKENSECRET"]

# Authenticate to Twitter
auth = tweepy.OAuthHandler(BG1_TWITTER_APIKEY, BG1_TWITTER_APIKEYSECRET)
auth.set_access_token(BG1_TWITTER_ACCESSTOKEN, BG1_TWITTER_ACCESSTOKENSECRET)

# Create API object 
api = tweepy.API(auth)
api

In [4]:
df_mun = pd.read_excel("BaseDatosAtlasMunicipalODSBolivia2020_Excel.xlsx", 3,header=2)
df_mun = df_mun.drop("Unnamed: 0", axis=1)
df_mun = df_mun.drop(columns=[i for i in df_mun.columns if i.startswith("Unnamed")])
df_mun = df_mun.replace(".", np.nan)

df_mun

,Código,Municipio,Departamento,Índice Municipal de Desarrollo Sostenible,Ranking del Índice de Bolivia,Población 2020,"Tasa de urbanización, 2012 (% de población)","Tasa de pobreza energética extrema, 2016 (% de viviendas)","Número de viviendas que consumen menos del 25% del límite de Tarifa Dignidad (210 kWh/año), 2016","Necesidades Básicas Insatisfechas, 2012 (% de población)",...,"Pérdida de Riqueza Absoluta de Especies, 2016-2018 (% de la pérdida nacional)","Tasa de homicidios registrados, promedio 2015-2017 (por 100,000 habitantes)","Número de homicidios registrados, 2015-2017","Capacidad de ejecución del presupuesto programado, 2017 (%)","Presupuesto Operativo Annual (POA) en Bs., 2017","Niños inscritos en el registro civil (< 5 años), 2012 (%)","Número de niños menores de 5 años que no están registrados en un registro civil, 2012","Proporción de ingresos municipales que provienen de impuestos locales, 2017 (%)","Inversión pública per cápita, 2017 (Bs./persona)","Inversión pública total, 2017 (Bs.)"
0,10101,Sucre,Chuquisaca,69.582053,5,295476,91.4,23.765676,17473.0,26.871265,...,0.000051,2.372924,20,86.205620,737627253,94.156170,1865,15.919012,671.308350,1.910114e+08
1,10102,Yotala,Chuquisaca,55.501551,75,10017,0.0,45.806933,1229.0,54.406515,...,0.000000,0.000000,0,65.516520,21577261,95.684114,47,3.390278,478.432770,4.746053e+06
2,10103,Poroma,Chuquisaca,35.662371,339,18947,0.0,96.050020,4839.0,95.943875,...,0.000000,1.811332,1,78.814679,31727744,85.178441,407,0.027649,475.135376,8.806159e+06
3,10201,Azurduy,Chuquisaca,37.926692,333,10948,0.0,86.848341,2199.0,87.749537,...,0.004009,0.000000,0,75.650322,32003762,88.009889,194,0.100616,997.513977,1.093475e+07
4,10202,Tarvita,Chuquisaca,39.974894,327,14688,0.0,86.095629,2935.0,86.932875,...,0.004071,0.000000,0,83.113015,32967137,85.707269,291,0.151069,625.205200,9.179263e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,90401,Santa Rosa,Pando,48.207247,217,2506,0.0,NaN,NaN,73.299398,...,0.170120,0.000000,0,79.593074,11766583,84.010152,63,0.000000,1056.830566,2.648417e+06
335,90402,Ingavi,Pando,43.623216,295,1840,0.0,NaN,NaN,86.695545,...,0.062554,0.000000,0,81.806600,11270615,75.075988,82,0.000000,2481.558594,4.451916e+06
336,90501,Nueva Esperanza,Pando,46.017078,273,2422,0.0,NaN,NaN,87.916431,...,0.120902,0.000000,0,65.138865,13017445,85.714286,43,0.000000,1290.759766,3.024250e+06
337,90502,Villa Nueva-Loma Alta,Pando,43.039756,305,3209,0.0,NaN,NaN,86.611630,...,0.059590,0.000000,0,76.906151,14426911,85.249042,77,0.000000,1308.056641,4.088985e+06


In [5]:
lista_indicadores_mun = list(df_mun.loc[:,"Índice Municipal de Desarrollo Sostenible":].columns)

indicator_dict = {}
for i,j in enumerate(lista_indicadores_mun):
    filename = str(i) + "_" + ''.join([ s[0] for s in j.split() ])
    indicator_dict.update({filename + ".png" : j})

indicator_dict

{'0_ÍMdDS.png': 'Índice Municipal de Desarrollo Sostenible',
 '1_RdÍdB.png': 'Ranking del Índice de Bolivia',
 '2_P2.png': 'Población 2020',
 '3_Tdu2(dp.png': 'Tasa de urbanización, 2012 (% de población)',
 '4_Tdpee2(dv.png': 'Tasa de pobreza energética extrema, 2016 (% de viviendas)',
 '5_Ndvqcmd2dldTD(k2.png': 'Número de viviendas que consumen menos del 25% del límite de Tarifa Dignidad (210 kWh/año), 2016',
 '6_NBI2(dp.png': 'Necesidades Básicas Insatisfechas, 2012 (% de población)',
 '7_Pcnbi2.png': 'Población con necesidades básicas insatisfechas, 2012',
 '8_ÍdPM2.png': 'Índice de Pobreza Multidimensional, 2012',
 '9_Ndhcfe4omd(92.png': 'Número de hogares con falencias en 4 o más dimensiones (de 9), 2012',
 '10_Aal3sb2(dh.png': 'Acceso a los 3 servicios básicos, 2012 (% de hogares)',
 '11_Ndhqcdamudltsb(ays2.png': 'Número de hogares que carecen de al menos uno de los tres servicios básicos (electricidad, agua y saneamiento), 2012',
 '12_Dcen(5a2(.png': 'Desnutrición crónica en niñ

# Post random sdsn mapa municipal

In [6]:
def post_sdsn_map():

    # map list
    map_list = os.listdir("mapas_municipales")

    # random map
    chosen_map = random.choice(map_list)

    # img path & upload media
    img_path = f"mapas_municipales/{chosen_map}"

    #get info
    indicador = indicator_dict[chosen_map]
    indicador_data = df_mun[[indicador, "Municipio"]].sort_values(indicador, ascending=False).dropna()
    # display(indicador_data)
    mayor = indicador_data.iloc[0]
    mayor_str = f"Municipio más álto: {mayor['Municipio']} con {mayor[indicador].round(2)}" 
    # display(mayor_str)
    menor = indicador_data.iloc[-1]
    menor_str = f"Municipio más bajo: {menor['Municipio']} con {menor[indicador].round(2)}" 
    # display(menor_str)


    # MESSAGE 
    MESSAGE = f"""
MAPA DEL DÍA:
{indicator_dict[chosen_map]} 

{mayor_str}
{menor_str}

(Fuente: SDSN Bolivia)
"""
    print(MESSAGE)


    # TWEET
    media_id = api.media_upload(img_path).media_id
    api.update_status(status = MESSAGE, media_ids = [media_id])
    

    print('TWEET SUCCESFUL!! ')
    return


post_sdsn_map()


MAPA DEL DÍA:
Población sin cobertura de energía eléctrica, 2012 

Municipio más álto: El Alto con 47166.49
Municipio más bajo: Huachacalla con 51.11

(Fuente: SDSN Bolivia)

TWEET SUCCESFUL!! 
